In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from geopy.distance import geodesic
from openrouteservice import Client
import openrouteservice.convert as convert
import time
from shapely.geometry import Point
from shapely.ops import unary_union

In [2]:
municipios_vega_baja = [
    "Torrevieja", "Orihuela", "Guardamar del Segura",
    "Los Montesinos", "Pilar de la Horadada", "Rojales", "San Miguel de Salinas"
]

In [3]:
# Obtener polígonos de cada municipio
gdfs = []
for municipio in municipios_vega_baja:
    gdf = ox.geocoder.geocode_to_gdf(f"{municipio}, Alicante, Spain")
    gdfs.append(gdf)

# Unir todos los municipios en un solo polígono (la comarca)
comarca = gpd.GeoDataFrame(geometry=[unary_union([g.geometry[0] for g in gdfs])], crs=gdfs[0].crs)

In [5]:
# Usar cualquier tag de POIs, o simplemente {"amenity": True} para muchos tipos
pois = ox.features.features_from_polygon(
    comarca.geometry.iloc[0],
    tags={"amenity": True}
)

In [79]:
pois = pois[['geometry', 'amenity', 'name', 'operator', 'opening_hours', 
             'source', 'website', 'religion', 'denomination', 'cuisine',
             'capacity', 'takeaway']]

In [4]:
pois = gpd.read_file("pois_vega_baja.geojson")

In [44]:
conteo = pois.groupby('amenity').size().reset_index(name='count').sort_values(by='count', ascending=False)
conteo

,amenity,count
16,parking,2896
18,restaurant,652
2,bar_cafe_pub,470
1,bank,102
13,lugares_de_culto,76
8,fast_food,58
0,atm,51
9,fountain,34
11,ice_cream,25
19,taxi,22


In [31]:
pois = pois[~((pois['amenity'] == 'townhall') & (pois['name'].isnull()))]

In [43]:
pois = pois[pois['amenity'] != 'bicycle_parking']

In [40]:
pois.loc[pois['amenity'] == 'Centro social', 'amenity'] = 'centros_sociales'

In [45]:
pois.to_file("pois_vega_baja.geojson", driver="GeoJSON")

In [17]:
pois['amenity'].unique()

array(['parking', 'restaurant', 'Lugar de culto', 'bar_cafe_pub', 'bank',
       'cinema', 'fast_food', 'taxi', 'theatre', 'townhall', 'gambling',
       'police', 'library', 'car_rental', 'atm', 'arts_centre',
       'bicycle_rental', 'fountain', 'ice_cream', 'ferry_terminal',
       'food_court', 'Centro social', 'marketplace', 'nightclub',
       'biergarten', 'bus_station', 'brothel', 'swingerclub',
       'bicycle_parking', 'casino', 'bbq', 'boat_rental', 'surf_school',
       'festival_grounds'], dtype=object)

In [46]:
pois.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4519 entries, 0 to 4600
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   element        4519 non-null   object  
 1   id             4519 non-null   int64   
 2   amenity        4519 non-null   object  
 3   name           1530 non-null   object  
 4   operator       101 non-null    object  
 5   opening_hours  124 non-null    object  
 6   source         15 non-null     object  
 7   website        88 non-null     object  
 8   religion       75 non-null     object  
 9   denomination   53 non-null     object  
 10  cuisine        278 non-null    object  
 11  capacity       311 non-null    object  
 12  takeaway       45 non-null     object  
 13  geometry       4519 non-null   geometry
dtypes: geometry(1), int64(1), object(12)
memory usage: 529.6+ KB


In [187]:
import folium
# Crear un color distinto por tipo de amenity
import itertools
import matplotlib.pyplot as plt

from folium import Marker
from folium.plugins import MarkerCluster

# Leer el GeoJSON
pois = gpd.read_file("pois_vega_baja.geojson")

# Centrar el mapa
center = pois.geometry.unary_union.centroid.coords[0]
m = folium.Map(location=[center[1], center[0]], zoom_start=11)

# Agrupar marcadores para mejor rendimiento visual
marker_cluster = MarkerCluster().add_to(m)

C:\Users\danim\AppData\Local\Temp\ipykernel_14596\1797652602.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = pois.geometry.unary_union.centroid.coords[0]


In [184]:
# Mapa de amenity a iconos de FontAwesome
amenity_icon_map = {
    "restaurant": "cutlery",
    "cafe": "coffee",
    "bank": "university",
    "atm": "credit-card",
    "parking": "parking",
    "Lugar de culto": "place-of-worship",
    "toilets": "restroom",
    "library": "book",
    "bar": "beer",
    "pub": "beer",
    "police": "shield-alt"
}

# Por defecto
default_icon = "map-marker"


In [ ]:
for _, row in pois.iterrows():
    if row.geometry.geom_type != "Point":
        continue

    amenity = row.get("amenity", "unknown")
    name = row.get("name", "Sin nombre")
    icon_name = amenity_icon_map.get(amenity, default_icon)

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(f"{amenity} - {name}", max_width=250),
        icon=folium.Icon(icon=icon_name, prefix="fa")
    ).add_to(marker_cluster)


m